In [ ]:
#default_exp foodc 

In [ ]:
# export
import tensorflow as tf
import ktrain
from ktrain import vision as vis
import pandas as pd
import numpy as np
import skimage.io
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [ ]:
#export

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Config

In [ ]:
#export

freeze_range = 177
learning_rate = 1e-5
rotation_range = 45
featurewise_center = False
featurewise_std_normalization =  False
zoom_range = 0.33
width_shift_range = 0.3
height_shift_range = 0.3
shear_range = 30

In [ ]:
#export

augmentations = vis.get_data_aug(
    horizontal_flip=True,
    rotation_range=rotation_range,
    featurewise_center=featurewise_center,
    featurewise_std_normalization=featurewise_std_normalization,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    shear_range=shear_range
)

In [ ]:
%matplotlib inline
%cd ..

In [ ]:
# export
TRAIN_PATH = 'data/train/train_images'
CSV_PATH = 'data/train.csv'

le = preprocessing.LabelEncoder()
train = pd.read_csv(CSV_PATH)
targets = le.fit_transform(train['ClassName'])
ntrain = train
ntrain['ClassName'] = targets

In [ ]:
#export
(train_data, val_data, preproc) = vis.images_from_csv(
    CSV_PATH,
    'ImageId',
    label_columns=['ClassName'],
    directory=TRAIN_PATH,
    random_state=0,
    data_aug=augmentations
)

In [ ]:
vis.show_random_images(TRAIN_PATH) 

In [ ]:
train_data[0][1].shape

In [ ]:
train = pd.read_csv(CSV_PATH)
num = train['ClassName'].value_counts()
classes = train['ClassName'].unique()
print('there are', len(classes), 'classes')
print()
print("Percentage of each class")
for cl in classes:
  print(cl,'\t',num[cl]/train.shape[0]*100,"%")

In [ ]:
imgs = train.loc[train['ClassName'] == 'bread-french-white-flour']
plt.figure(figsize=(10,10))
for i in range(imgs[:16].shape[0]):
    path = imgs.iloc[i]['ImageId']
    image = skimage.io.imread(os.path.join(TRAIN_PATH,path))
    plt.subplot(4,4,i+1)
    plt.axis('off')
    plt.imshow(image)

In [ ]:
vis.print_image_classifiers()

In [ ]:
#export

model = vis.image_classifier('pretrained_resnet50', train_data, val_data)
learner = ktrain.get_learner(model=model, train_data=train_data, val_data=val_data, 
                             workers=8, use_multiprocessing=True, batch_size=32)

learner.freeze(freeze_range=freeze_range)

In [ ]:
model.summary()

In [ ]:
len(model.layers)

In [ ]:
learner.lr_find()

In [ ]:
learner.lr_plot()

In [ ]:
#export
learner.autofit(lr=learning_rate)

In [ ]:
learner.save('model.h5')